In [2]:
import numpy as np
import cv2

In [3]:
path_img = './pic/main.jpg'
img = cv2.imread(path_img)
draw_img = img

blur = cv2.GaussianBlur(img, (3, 3), 0)
h, w = img.shape[:2]

In [6]:
# Morphological gradient

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
gradient = cv2.morphologyEx(blur, cv2.MORPH_GRADIENT, kernel)

cv2.imshow('Morphological gradient', gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# Binarize gradient

lowerb = np.array([0, 0, 0])
upperb = np.array([11, 11, 11])
binary = cv2.inRange(gradient, lowerb, upperb)

cv2.imshow('Binarized gradient', binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# Flood fill from the edges to remove edge object

for row in range(h):
    if binary[row, 0] == 255:
        cv2.floodFill(binary, None, (0, row), 0)
    if binary[row, w-1] == 255:
        cv2.floodFill(binary, None, (w-1, row), 0)

for col in range(w):
    if binary[0, col] == 255:
        cv2.floodFill(binary, None, (col, 0), 0)
    if binary[h-1, col] == 255:
        cv2.floodFill(binary, None, (col, h-1), 0)

cv2.imshow('Filled binary gradient', binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Cleaning up mask

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

foreground = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
foreground = cv2.morphologyEx(foreground, cv2.MORPH_CLOSE, kernel)

cv2.imshow('Cleanup up crystal foreground mask', foreground)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# Creating background and unknown mask for labeling

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (17, 17))
background = cv2.dilate(foreground, kernel, iterations=3)
unknown = cv2.subtract(background, foreground)

cv2.imshow('Background', background)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Watershed

ret, markers = cv2.connectedComponents(foreground)
markers += 1  # Add one to all labels so that background is 1, not 0
markers[unknown==255] = 0  # mark the region of unknown with zero
markers = cv2.watershed(img, markers)
print(ret)

766


In [13]:
# Assign the markers a hue between 0 and 179

hue_markers = np.uint8(179*np.float32(markers)/np.max(markers))
blank_channel = 255*np.ones((h, w), dtype=np.uint8)

marker_img = cv2.merge([hue_markers, blank_channel, blank_channel])
marker_img = cv2.cvtColor(marker_img, cv2.COLOR_HSV2BGR)
marker_img[hue_markers == 0] = 0

cv2.imshow('Colored markers', marker_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# Label the original image with the watershed markers

labeled_img = img.copy()
labeled_img[markers>1] = ma144116.5 rker_img[markers>1]  # 1 is background color
labeled_img = cv2.addWeighted(img, 0.5, labeled_img, 0.5, 0)

cv2.imshow('watershed_result.png', labeled_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
cv2.imwrite('./pic/output_preprocessing.png', labeled_img)

True